### ChatDB的Prompt

chatdb.py的generate_chat_responses()，get_steps_from_response()，token限制和历史上下文没有拿过来

In [24]:
from langchain.prompts import PromptTemplate

In [25]:


tab_fruits = """
CREATE TABLE fruits (
    fruit_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    fruit_name TEXT NOT NULL,
    selling_price REAL,
    stock_quantity INTEGER,
    fruit_type TEXT,
    shelf_life INTEGER
);
"""

tab_customers = """
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    first_name TEXT,
    last_name TEXT,
    phone_number TEXT,
    email TEXT
);
"""

tab_suppliers = """
CREATE TABLE suppliers (
    supplier_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    supplier_name TEXT NOT NULL,
    contact_number TEXT,
    email TEXT
);
"""

tab_sales = """
CREATE TABLE sales (
    sale_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    customer_id INTEGER,
    sale_date DATE,
    total_price REAL,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""

tab_sale_items = """
CREATE TABLE sale_items (
    sale_item_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    sale_id INTEGER NOT NULL,
    fruit_id INTEGER NOT NULL,
    quantity_sold INTEGER,
    price_per_item REAL,
    item_total_price REAL,
    FOREIGN KEY (sale_id) REFERENCES sales(sale_id),
    FOREIGN KEY (fruit_id) REFERENCES fruits(fruit_id)
);
"""

tab_purchases = """
CREATE TABLE purchases (
    purchase_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    supplier_id INTEGER NOT NULL,
    purchase_date DATE,
    total_cost REAL,
    FOREIGN KEY (supplier_id) REFERENCES suppliers(supplier_id)
);
"""

tab_purchase_items = """
CREATE TABLE purchase_items (
    purchase_item_id INTEGER PRIMARY KEY AUTO_INCREMENT,
    purchase_id INTEGER NOT NULL,
    fruit_id INTEGER NOT NULL,
    quantity_purchased INTEGER,
    cost_per_item REAL,
    item_total_cost REAL,
    FOREIGN KEY (purchase_id) REFERENCES purchases(purchase_id),
    FOREIGN KEY (fruit_id) REFERENCES fruits(fruit_id)
);
"""

tables = [tab_fruits, tab_customers, tab_suppliers, tab_sales, tab_sale_items, tab_purchases, tab_purchase_items]


In [26]:
import re


def get_table_info(create_table_sql):
    # Regex to match 'CREATE TABLE' followed by the table name
    pattern = r"CREATE TABLE (\w+)"
    match = re.search(pattern, create_table_sql, re.IGNORECASE)
    table_name = match.group(1)

    # Regex to match the column definitions starting with a lowercase letter
    # This pattern matches a word at the start of the line, followed by a space, followed by another word
    pattern = r"^\s*([a-z]\w*)\s+(\w+)"
    matches = re.findall(pattern, create_table_sql, re.MULTILINE)

    # The matches list now contains tuples with the column name and data type
    column_names = []
    column_types = []
    for column_name, data_type in matches:
        column_names.append(column_name)
        column_types.append(data_type)

    return table_name, column_names, column_types


def get_database_info(tables):
    database_info = dict()
    for tab in tables:
        table_name, column_names, column_types = get_table_info(tab)
        database_info[table_name] = {
            "column_names": column_names,
            "column_types": column_types,
        }
    return database_info


database_info = get_database_info(tables)
table_details = "\n".join(tables)

In [27]:
def init_system_msg():
    sys_temp = """
You are ChatDB, a powerful AI assistant, a variant of ChatGPT that can utilize databases as external symbolic memory. \
You are an expert in databases, proficient in SQL statements and can use the database to help users. \
The details of tables in the database are delimited by triple quotes.
\"\"\"
{table_details}
\"\"\"
"""
    sys_prompt = PromptTemplate(
        template=sys_temp,
        input_variables=[],
        partial_variables={"table_details": table_details, }
    )
    sys_prompt_str = sys_prompt.format()
    return sys_prompt_str

In [28]:
sys_prompt = init_system_msg()
sys_prompt

'\nYou are ChatDB, a powerful AI assistant, a variant of ChatGPT that can utilize databases as external symbolic memory. You are an expert in databases, proficient in SQL statements and can use the database to help users. The details of tables in the database are delimited by triple quotes.\n"""\n\nCREATE TABLE fruits (\n    fruit_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    fruit_name TEXT NOT NULL,\n    selling_price REAL,\n    stock_quantity INTEGER,\n    fruit_type TEXT,\n    shelf_life INTEGER\n);\n\n\nCREATE TABLE customers (\n    customer_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    first_name TEXT,\n    last_name TEXT,\n    phone_number TEXT,\n    email TEXT\n);\n\n\nCREATE TABLE suppliers (\n    supplier_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    supplier_name TEXT NOT NULL,\n    contact_number TEXT,\n    email TEXT\n);\n\n\nCREATE TABLE sales (\n    sale_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    customer_id INTEGER,\n    sale_date DATE,\n    total_price REAL,\n    FOREIGN KE

In [29]:
eg_purchase = """
USER INPUT: On 2023-01-22, the shop purchased 100kg banana from supplier 'ABC' (contact number: 67543, email: abc_sup@gmail.com) at 1.2 dollar/kg and planed sell at 1.8 dollar/kg. Banana's fruit type is berry and shelf life is 15 days.
ANSWER:
```
Step1: Insert supplier 'ABC' if not exists
`INSERT INTO suppliers (supplier_name, contact_number, email)
SELECT 'ABC', '67543', 'abc_sup@gmail.com'
WHERE NOT EXISTS (SELECT 1 FROM suppliers WHERE supplier_name = 'ABC');`

Step2: Insert fruit (set the selling price to NULL and stock quantity to 0) if not exists
`INSERT INTO fruits (fruit_name, selling_price, stock_quantity, fruit_type, shelf_life)
SELECT 'banana', NULL, 0, 'berry', 15
WHERE NOT EXISTS (SELECT 1 FROM fruits WHERE fruit_name = 'banana');`

Step3: Insert purchase
`INSERT INTO purchases (supplier_id, purchase_date, total_cost)
VALUES ((SELECT supplier_id FROM suppliers WHERE supplier_name = 'ABC'), '2023-01-22', 100 * 1.2);`

Step4: Insert purchase item
`INSERT INTO purchase_items (purchase_id, fruit_id, quantity_purchased, cost_per_item, item_total_cost)
VALUES ((SELECT MAX(purchase_id) FROM purchases), (SELECT fruit_id FROM fruits WHERE fruit_name = 'banana'), 100, 1.2, 100 * 1.2);`

Step5: Update the stock quantity of banana
`UPDATE fruits
SET stock_quantity = stock_quantity + 100
WHERE fruit_name = 'banana';`

Step6: Update the selling price of banana if given new selling price
`UPDATE fruits
SET selling_price = 1.8
WHERE fruit_name = 'banana';`
```
"""

eg_ask_sale = """
USER INPUT: Who bought 100kg apple on 2010-03-27 and what is he/she name, detailed information and costumer id?
ANSWER:
```
Step1: Retrieve the customer information who made the purchase
`SELECT c.customer_id, c.first_name, c.last_name, c.phone_number, c.email
FROM customers c
JOIN sales s ON c.customer_id = s.customer_id
JOIN sale_items si ON s.sale_id = si.sale_id
JOIN fruits f ON si.fruit_id = f.fruit_id
WHERE f.fruit_name = 'apple' AND si.quantity_sold = 100 AND s.sale_date = '2010-03-27';`
```
"""

eg_if_new_customer_sale = """
USER INPUT: A customer named 'Chenzhuang Du' with a phone number as 120056 and e-mail as chenzhuang@gmail.com bought 10kg apple and 5kg pear on 2010-03-27.
ANSWER:
```
Step1: Insert customer 'Chenzhuang Du' if not exists
`INSERT INTO customers (first_name, last_name, phone_number, email)
SELECT 'Chenzhuang', 'Du', '120056', 'chenzhuang@gmail.com'
WHERE NOT EXISTS (SELECT 1 FROM customers WHERE phone_number = '120056');`

Step2: Insert sale
`INSERT INTO sales (customer_id, sale_date, total_price)
VALUES ((SELECT customer_id FROM customers WHERE phone_number = '120056'), '2010-03-27', (SELECT selling_price FROM fruits WHERE fruit_name = 'apple') * 10 + (SELECT selling_price FROM fruits WHERE fruit_name = 'pear') * 5);`

Step3: Insert sale item
`INSERT INTO sale_items (sale_id, fruit_id, quantity_sold, price_per_item, item_total_price)
VALUES ((SELECT MAX(sale_id) FROM sales), (SELECT fruit_id FROM fruits WHERE fruit_name = 'apple'), 10, (SELECT selling_price FROM fruits WHERE fruit_name = 'apple'), (SELECT selling_price FROM fruits WHERE fruit_name = 'apple') * 10),
((SELECT MAX(sale_id) FROM sales), (SELECT fruit_id FROM fruits WHERE fruit_name = 'pear'), 5, (SELECT selling_price FROM fruits WHERE fruit_name = 'pear'), (SELECT selling_price FROM fruits WHERE fruit_name = 'pear') * 5);`

Step4: Update the stock quantity of apple and pear
`UPDATE fruits
SET stock_quantity = CASE
    WHEN fruit_name = 'apple' THEN stock_quantity - 10
    WHEN fruit_name = 'pear' THEN stock_quantity - 5
    ELSE stock_quantity
END
WHERE fruit_name IN ('apple', 'pear');`
```
"""

eg_delete_pro = """
USER INPUT: Because the customer returned the goods, roll back a sales record which is made by the customer with customer id as 8 on 2023-01-29.
ANSWER:
```
Step1: Find the sale_id for this customer on this date
`SELECT sale_id FROM sales WHERE customer_id = 8 AND sale_date = '2023-01-29';`

Step2: Get all the fruit_id and quantity_sold for this sale, replace <sale_id> with the results from the previous queries
`SELECT fruit_id, quantity_sold FROM sale_items WHERE sale_id = <sale_id>;`

Step3: Increase the stock_quantity for each fruit sold in this sale, replace <quantity_sold> <fruit_id> with the results from the previous queries
`UPDATE fruits
SET stock_quantity = stock_quantity + <quantity_sold>
WHERE fruit_id = <fruit_id>;`

Step4: Delete the sale items for this sale, replace <sale_id> with the results from the previous queries
`DELETE FROM sale_items WHERE sale_id = <sale_id>;`

Step5: Delete the sale record, replace <sale_id> with the results from the previous queries
`DELETE FROM sales WHERE sale_id = <sale_id>;`
```
"""


egs = [eg_ask_sale, eg_purchase, eg_if_new_customer_sale, eg_delete_pro]

In [30]:
prompt_ask_steps_temp = """
Please tell me what standard SQL statements should I use in order to respond to the "USER INPUT". \
If it needs multiple SQL operations on the database, please list them step by step concisely. \
If there is no need to use the database, reply to the "USER INPUT" directly.
The output should be a markdown code snippet formatted in the following schema, \
including the leading and trailing "\`\`\`" and "\`\`\`":
```
Step1: <Description of first step>
SQL command for step1

Step2: <Description of second step>
SQL command for step2

......
```
Here are some examples:
{egs}

USER INPUT: {user_inp}
ANSWER:
"""

prompt_ask_steps = PromptTemplate(
        template=prompt_ask_steps_temp,
        input_variables=["user_inp"],
        partial_variables={
            "egs": '\n'.join(egs),
        }
    )

prompt_ask_steps_no_egs = PromptTemplate(
        template=prompt_ask_steps_temp,
        input_variables=["user_inp"],
        partial_variables={
            "egs": ""
        }
    )

In [40]:
# user_prompt = prompt_ask_steps.format(user_inp="How many apples does the customer named 'Chenzhuang Du' bought on 2010-03-27")
user_prompt = prompt_ask_steps.format(user_inp="A customer named 'Chenzhuang Du' with a phone number as 120056 and e-mail as chenzhuang@gmail.com bought 10kg apple and 5kg pear on 2010-03-27")
user_prompt

'\nPlease tell me what standard SQL statements should I use in order to respond to the "USER INPUT". If it needs multiple SQL operations on the database, please list them step by step concisely. If there is no need to use the database, reply to the "USER INPUT" directly.\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`" and "\\`\\`\\`":\n```\nStep1: <Description of first step>\nSQL command for step1\n\nStep2: <Description of second step>\nSQL command for step2\n\n......\n```\nHere are some examples:\n\nUSER INPUT: Who bought 100kg apple on 2010-03-27 and what is he/she name, detailed information and costumer id?\nANSWER:\n```\nStep1: Retrieve the customer information who made the purchase\n`SELECT c.customer_id, c.first_name, c.last_name, c.phone_number, c.email\nFROM customers c\nJOIN sales s ON c.customer_id = s.customer_id\nJOIN sale_items si ON s.sale_id = si.sale_id\nJOIN fruits f ON si.fruit_id = f.frui

In [41]:
import traceback
import time
import json
import requests
from log_tools import slogger
import config


def get_openai_data_raw(messages, model='gpt-3.5-turbo',temperature=0.0):
    content = None
    # url = f'{api_host}/chat/completions'
    url = f'{config.OPENAI_API_HOST_USE}/chat/completions'
    headers = {'Authorization': f'Bearer {config.OPENAI_API_KEY}',
               'Content-Type': 'application/json'}

    data = {
        "model": model,
        "temperature": temperature,
        "messages": messages
    }
    slogger.info(f"get_openai_data: model:{model}")
    try:
        response = requests.post(url, data=json.dumps(data), headers=headers, timeout=300)  # 不然服务器会卡死
        slogger.info(response)
        slogger.info(f"response.status_code:{response.status_code}, response.text:{response.text}")
        if response.status_code == 200:
            if "That model is currently overloaded with other requests" in response.text:
                slogger.info(f"openai overloaded, get_openai_data retrying:{url}")
                time.sleep(3)
                response = requests.post(url, data=json.dumps(data), headers=headers, timeout=300)
                slogger.info(response)
                slogger.info(f"response.status_code:{response.status_code}, response.text:{response.text}")
            json_dict = response.json()  # 解析JSON
            slogger.info(f"json_dict:{json_dict}")
            # 从JSON字符串中提取内容字段
            content = json_dict['choices'][0]['message']['content']  # 获取指定字段
            slogger.info(f"content:{content}")
        else:
            slogger.info("请求失败")
    except Exception as e:
        traceback.print_exc()
        slogger.info(f"error:{e}")
    return content
    # 处理响应结果


In [42]:
    # Set up the prompt
messages = [{
    'role': 'system',
    'content': sys_prompt
}, {
    "role":
    "user",
    "content":user_prompt,
}]
messages

[{'role': 'system',
  'content': '\nYou are ChatDB, a powerful AI assistant, a variant of ChatGPT that can utilize databases as external symbolic memory. You are an expert in databases, proficient in SQL statements and can use the database to help users. The details of tables in the database are delimited by triple quotes.\n"""\n\nCREATE TABLE fruits (\n    fruit_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    fruit_name TEXT NOT NULL,\n    selling_price REAL,\n    stock_quantity INTEGER,\n    fruit_type TEXT,\n    shelf_life INTEGER\n);\n\n\nCREATE TABLE customers (\n    customer_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    first_name TEXT,\n    last_name TEXT,\n    phone_number TEXT,\n    email TEXT\n);\n\n\nCREATE TABLE suppliers (\n    supplier_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    supplier_name TEXT NOT NULL,\n    contact_number TEXT,\n    email TEXT\n);\n\n\nCREATE TABLE sales (\n    sale_id INTEGER PRIMARY KEY AUTO_INCREMENT,\n    customer_id INTEGER,\n    sale_date DATE,\n    

In [38]:
res = get_openai_data_raw(messages=messages, model='gpt-3.5-turbo',temperature=0.0)
print(res)

2023-07-13 11:53:49,230 - 323910328.py - [line]:21 - INFO - get_openai_data: model:gpt-3.5-turbo
2023-07-13 11:53:53,222 - 323910328.py - [line]:24 - INFO - <Response [200]>
2023-07-13 11:53:53,222 - 323910328.py - [line]:25 - INFO - response.status_code:200, response.text:{
  "id": "chatcmpl-7bhgnVFrGF0YGO7PoQe9jargXyIco",
  "object": "chat.completion",
  "created": 1689220433,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```\nStep1: Retrieve the quantity of apples bought by the customer\nSELECT si.quantity_sold\nFROM customers c\nJOIN sales s ON c.customer_id = s.customer_id\nJOIN sale_items si ON s.sale_id = si.sale_id\nJOIN fruits f ON si.fruit_id = f.fruit_id\nWHERE c.first_name = 'Chenzhuang' AND c.last_name = 'Du' AND f.fruit_name = 'apple' AND s.sale_date = '2010-03-27';\n```"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1792,
    "completion_tok

"```\nStep1: Retrieve the quantity of apples bought by the customer\nSELECT si.quantity_sold\nFROM customers c\nJOIN sales s ON c.customer_id = s.customer_id\nJOIN sale_items si ON s.sale_id = si.sale_id\nJOIN fruits f ON si.fruit_id = f.fruit_id\nWHERE c.first_name = 'Chenzhuang' AND c.last_name = 'Du' AND f.fruit_name = 'apple' AND s.sale_date = '2010-03-27';\n```"

In [39]:
res = get_openai_data_raw(messages=messages, model='gpt-3.5-turbo-16k',temperature=0.0)
print(res)

2023-07-13 11:56:04,046 - 323910328.py - [line]:21 - INFO - get_openai_data: model:gpt-3.5-turbo-16k
2023-07-13 11:56:07,454 - 323910328.py - [line]:24 - INFO - <Response [200]>
2023-07-13 11:56:07,454 - 323910328.py - [line]:25 - INFO - response.status_code:200, response.text:{
  "id": "chatcmpl-7bhiymfIxaza2ANkXFR2bdXpfyzYo",
  "object": "chat.completion",
  "created": 1689220568,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```\nStep1: Retrieve the quantity of apples bought by the customer\nSELECT si.quantity_sold\nFROM customers c\nJOIN sales s ON c.customer_id = s.customer_id\nJOIN sale_items si ON s.sale_id = si.sale_id\nJOIN fruits f ON si.fruit_id = f.fruit_id\nWHERE c.first_name = 'Chenzhuang' AND c.last_name = 'Du' AND f.fruit_name = 'apple' AND s.sale_date = '2010-03-27';\n```"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1792,
    "comple

```
Step1: Retrieve the quantity of apples bought by the customer
SELECT si.quantity_sold
FROM customers c
JOIN sales s ON c.customer_id = s.customer_id
JOIN sale_items si ON s.sale_id = si.sale_id
JOIN fruits f ON si.fruit_id = f.fruit_id
WHERE c.first_name = 'Chenzhuang' AND c.last_name = 'Du' AND f.fruit_name = 'apple' AND s.sale_date = '2010-03-27';
```


In [43]:
res = get_openai_data_raw(messages=messages, model='gpt-3.5-turbo-16k',temperature=0.0)
print(res)

2023-07-13 11:56:55,440 - 323910328.py - [line]:21 - INFO - get_openai_data: model:gpt-3.5-turbo-16k
2023-07-13 11:57:03,255 - 323910328.py - [line]:24 - INFO - <Response [200]>
2023-07-13 11:57:03,255 - 323910328.py - [line]:25 - INFO - response.status_code:200, response.text:{
  "id": "chatcmpl-7bhjnRn40NliFUCBXytcRTf7vaor3",
  "object": "chat.completion",
  "created": 1689220619,
  "model": "gpt-3.5-turbo-16k-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "```\nStep1: Insert customer 'Chenzhuang Du' if not exists\n`INSERT INTO customers (first_name, last_name, phone_number, email)\nSELECT 'Chenzhuang', 'Du', '120056', 'chenzhuang@gmail.com'\nWHERE NOT EXISTS (SELECT 1 FROM customers WHERE phone_number = '120056');`\n\nStep2: Insert sale\n`INSERT INTO sales (customer_id, sale_date, total_price)\nVALUES ((SELECT customer_id FROM customers WHERE phone_number = '120056'), '2010-03-27', (SELECT selling_price FROM fruits WH

```
Step1: Insert customer 'Chenzhuang Du' if not exists
`INSERT INTO customers (first_name, last_name, phone_number, email)
SELECT 'Chenzhuang', 'Du', '120056', 'chenzhuang@gmail.com'
WHERE NOT EXISTS (SELECT 1 FROM customers WHERE phone_number = '120056');`

Step2: Insert sale
`INSERT INTO sales (customer_id, sale_date, total_price)
VALUES ((SELECT customer_id FROM customers WHERE phone_number = '120056'), '2010-03-27', (SELECT selling_price FROM fruits WHERE fruit_name = 'apple') * 10 + (SELECT selling_price FROM fruits WHERE fruit_name = 'pear') * 5);`

Step3: Insert sale item
`INSERT INTO sale_items (sale_id, fruit_id, quantity_sold, price_per_item, item_total_price)
VALUES ((SELECT MAX(sale_id) FROM sales), (SELECT fruit_id FROM fruits WHERE fruit_name = 'apple'), 10, (SELECT selling_price FROM fruits WHERE fruit_name = 'apple'), (SELECT selling_price FROM fruits WHERE fruit_name = 'apple') * 10),
((SELECT MAX(sale_id) FROM sales), (SELECT fruit_id FROM fruits WHERE fruit_name = 